In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import pymysql
import datetime
from prophet import Prophet

In [2]:
data = pd.read_csv('./닭고기가격.csv',index_col=0)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2538 entries, 1 to 3
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    2538 non-null   object 
 1   5_6호    2538 non-null   float64
 2   7_8호    2538 non-null   float64
 3   9_10호   2538 non-null   float64
 4   11호     2538 non-null   float64
 5   12호     2538 non-null   float64
 6   13_16호  2538 non-null   float64
dtypes: float64(6), object(1)
memory usage: 158.6+ KB


In [4]:
data['date'] = pd.to_datetime(data['date'])
data

,date,5_6호,7_8호,9_10호,11호,12호,13_16호
1,2016-01-02,3933.0,3767.0,3462.0,3326.0,3089.0,3050.0
2,2016-01-03,3933.0,3767.0,3462.0,3326.0,3089.0,3050.0
3,2016-01-04,3933.0,3767.0,3462.0,3326.0,3089.0,3050.0
4,2016-01-05,3933.0,3767.0,3462.0,3326.0,3089.0,3050.0
5,2016-01-06,3933.0,3767.0,3462.0,3326.0,3089.0,3050.0
...,...,...,...,...,...,...,...
7,2022-11-16,4767.0,4561.0,4231.0,4062.0,3814.0,3764.0
6,2022-11-17,4767.0,4561.0,4231.0,4062.0,3814.0,3764.0
5,2022-11-18,4767.0,4561.0,4231.0,4062.0,3814.0,3764.0
4,2022-11-19,4767.0,4561.0,4231.0,4062.0,3814.0,3764.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2538 entries, 1 to 3
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2538 non-null   datetime64[ns]
 1   5_6호    2538 non-null   float64       
 2   7_8호    2538 non-null   float64       
 3   9_10호   2538 non-null   float64       
 4   11호     2538 non-null   float64       
 5   12호     2538 non-null   float64       
 6   13_16호  2538 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 158.6 KB


In [13]:
data = data.drop_duplicates(subset=['date'],keep='first')
data.info()
data.to_csv('./중복확인.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2518 entries, 1 to 0
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2518 non-null   datetime64[ns]
 1   5_6호    2518 non-null   float64       
 2   7_8호    2518 non-null   float64       
 3   9_10호   2518 non-null   float64       
 4   11호     2518 non-null   float64       
 5   12호     2518 non-null   float64       
 6   13_16호  2518 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 157.4 KB


In [6]:
mydb = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '1234',
    db = 'price',
    charset='utf8'
)

cursor = mydb.cursor(pymysql.cursors.DictCursor)
cursor.execute("SELECT * FROM price.chicken ORDER BY 1 DESC LIMIT 6")
result = cursor.fetchall()

for row in result :
    print(row)


{'date': '2022-11-26', '요일': '토', '5_6호': 4933, '7_8호': 4720, '9_10호': 4385, '11호': 4209, '12호': 3959, '13_16호': 3907}
{'date': '2022-11-25', '요일': '금', '5_6호': 4933, '7_8호': 4720, '9_10호': 4385, '11호': 4209, '12호': 3959, '13_16호': 3907}
{'date': '2022-11-24', '요일': '목', '5_6호': 4933, '7_8호': 4720, '9_10호': 4385, '11호': 4209, '12호': 3959, '13_16호': 3907}
{'date': '2022-11-23', '요일': '수', '5_6호': 4933, '7_8호': 4720, '9_10호': 4385, '11호': 4209, '12호': 3959, '13_16호': 3907}
{'date': '2022-11-22', '요일': '화', '5_6호': 4933, '7_8호': 4720, '9_10호': 4385, '11호': 4209, '12호': 3959, '13_16호': 3907}
{'date': '2022-11-21', '요일': '월', '5_6호': 4767, '7_8호': 4561, '9_10호': 4231, '11호': 4062, '12호': 3814, '13_16호': 3764}


In [7]:
# 조회 결과 데이터프레임으로 저장
df = pd.DataFrame(result)
df.drop(columns='요일',inplace=True)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date',ascending=True)

df

,date,5_6호,7_8호,9_10호,11호,12호,13_16호
5,2022-11-21,4767,4561,4231,4062,3814,3764
4,2022-11-22,4933,4720,4385,4209,3959,3907
3,2022-11-23,4933,4720,4385,4209,3959,3907
2,2022-11-24,4933,4720,4385,4209,3959,3907
1,2022-11-25,4933,4720,4385,4209,3959,3907
0,2022-11-26,4933,4720,4385,4209,3959,3907


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 5 to 0
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    6 non-null      datetime64[ns]
 1   5_6호    6 non-null      int64         
 2   7_8호    6 non-null      int64         
 3   9_10호   6 non-null      int64         
 4   11호     6 non-null      int64         
 5   12호     6 non-null      int64         
 6   13_16호  6 non-null      int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 384.0 bytes


In [9]:
import datetime

tomorrow = df['date'][-1:]  + datetime.timedelta(days=1)
tomorrow = pd.DataFrame(tomorrow)

df = pd.merge(df,tomorrow,on='date',how='outer')
df = df.fillna(method='ffill')
df['date'] = pd.to_datetime(df['date'])
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    7 non-null      datetime64[ns]
 1   5_6호    7 non-null      float64       
 2   7_8호    7 non-null      float64       
 3   9_10호   7 non-null      float64       
 4   11호     7 non-null      float64       
 5   12호     7 non-null      float64       
 6   13_16호  7 non-null      float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 448.0 bytes


In [90]:
data = pd.concat((data,df),axis=0)
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by='date', ascending=True)
data = data[-2220:]
data.to_csv('./닭고기가격.csv')
data

,date,5_6호,7_8호,9_10호,11호,12호,13_16호
326,2016-11-22,3600.0,3450.0,3308.0,3179.0,3089.0,3050.0
327,2016-11-23,3600.0,3450.0,3308.0,3179.0,3089.0,3050.0
328,2016-11-24,3600.0,3450.0,3308.0,3179.0,3089.0,3050.0
329,2016-11-25,3600.0,3450.0,3308.0,3179.0,3089.0,3050.0
330,2016-11-26,3600.0,3450.0,3308.0,3179.0,3089.0,3050.0
...,...,...,...,...,...,...,...
0,2022-11-23,4933.0,4720.0,4385.0,4209.0,3959.0,3907.0
3,2022-11-24,4933.0,4720.0,4385.0,4209.0,3959.0,3907.0
4,2022-11-25,4933.0,4720.0,4385.0,4209.0,3959.0,3907.0
5,2022-11-26,4933.0,4720.0,4385.0,4209.0,3959.0,3907.0


In [18]:
# prophet은 ds,y라는 두가지 컬럼만 있어야 함(컬럼 이름 맞추기)
data_5_6 = data[['date', '5_6호']]
data_5_6 = data_5_6.rename(columns=({'date':'ds','5_6호':'y'}))

In [19]:
data_7_8 = data[['date', '7_8호']]
data_7_8 = data_7_8.rename(columns=({'date':'ds','7_8호':'y'}))
data_7_8.head()

,ds,y
326,2016-11-22,NaN
327,2016-11-23,NaN
328,2016-11-24,NaN
329,2016-11-25,NaN
330,2016-11-26,NaN


In [20]:
data_9_10 = data[['date', '9_10호']]
data_9_10 = data_9_10.rename(columns=({'date':'ds','9_10호':'y'}))
data_9_10.head()

,ds,y
326,2016-11-22,NaN
327,2016-11-23,NaN
328,2016-11-24,NaN
329,2016-11-25,NaN
330,2016-11-26,NaN


In [21]:
data_11 = data[['date', '11호']]
data_11 = data_11.rename(columns=({'date':'ds','11호':'y'}))
data_11.head()

,ds,y
326,2016-11-22,3179.0
327,2016-11-23,3179.0
328,2016-11-24,3179.0
329,2016-11-25,3179.0
330,2016-11-26,3179.0


In [22]:
data_12 = data[['date', '12호']]
data_12 = data_12.rename(columns=({'date':'ds','12호':'y'}))
data_12.head()

,ds,y
326,2016-11-22,3089.0
327,2016-11-23,3089.0
328,2016-11-24,3089.0
329,2016-11-25,3089.0
330,2016-11-26,3089.0


In [23]:
data_13_16 = data[['date', '13_16호']]
data_13_16 = data_13_16.rename(columns=({'date':'ds','13_16호':'y'}))
data_13_16.head()

,ds,y
326,2016-11-22,NaN
327,2016-11-23,NaN
328,2016-11-24,NaN
329,2016-11-25,NaN
330,2016-11-26,NaN


In [24]:
Bok_nal = pd.DataFrame({
'holiday': 'Bok_nal',
'ds': pd.to_datetime(['2016-07-17', '2016-07-27', '2016-08-16',
                      '2017-07-12', '2017-07-22', '2017-08-11',
                      '2018-07-17', '2018-07-27', '2018-08-16',
                      '2019-07-12', '2019-07-22', '2019-08-11',
                      '2020-07-16', '2020-07-26', '2020-08-15',
                      '2021-07-11', '2021-07-21', '2021-08-10',
                      '2022-07-16', '2022-07-26', '2022-08-15'
                      ]),
'lower_window': 0,
'upper_window': 1,
})

AI = pd.DataFrame({
'holiday': 'AI',
'ds': pd.to_datetime([
    '2016-11-16', '2016-11-22', '2016-11-23', '2016-11-26', 
    '2016-11-27', '2016-11-28', '2016-11-29', '2016-11-30', '2016-12-01',
    '2016-12-02', '2016-12-03', '2016-12-04', '2016-12-05', '2016-12-06', 
    '2016-12-07', '2016-12-08', '2016-12-09', '2016-12-10', '2016-12-11', 
    '2016-12-12', '2016-12-13', '2016-12-14', '2016-12-15', '2016-12-16',
    '2016-12-17', '2016-12-18', '2016-12-19', '2016-12-20', '2016-12-21', 
    '2016-12-22', '2016-12-23', '2016-12-24', '2016-12-25', '2016-12-26', 
    '2016-12-27', '2016-12-28', '2016-12-29', '2016-12-30',
    '2017-01-01', '2017-01-03', '2017-01-05', '2017-01-07', '2017-01-08', 
    '2017-01-09', '2017-01-10', '2017-01-12', '2017-01-14', '2017-01-20', 
    '2017-01-21', '2017-01-24', '2017-02-06', '2017-02-09', '2017-02-22', 
    '2017-02-27', '2017-03-01', '2017-03-02', '2017-03-03', '2017-03-06', 
    '2017-03-17', '2017-03-19', '2017-03-22', '2017-03-23', '2017-03-26', 
    '2017-03-27', '2017-03-29', '2017-04-02', '2017-06-03', '2018-01-03', 
    '2018-01-26', '2018-01-27', '2018-02-04', '2018-02-08', '2018-03-16', 
    '2018-03-17',
    '2020-12-06', '2020-12-12', '2020-12-14', '2020-12-16', '2020-12-21', 
    '2020-12-22', '2020-12-25', '2020-12-28', '2020-12-28', '2020-12-29',
    '2020-12-29', '2020-12-31', '2021-01-01', '2021-01-03', '2021-01-04',
    '2021-01-08', '2021-01-11', '2021-01-12', '2021-01-11', '2021-01-12',
    '2021-01-13', '2021-01-18', '2021-01-21', '2021-01-22', '2021-01-23',
    '2021-01-26', '2021-01-26', '2021-01-27', '2021-01-29', '2021-01-30',
    '2021-01-31', '2021-02-01', '2021-02-07', '2021-02-09', '2021-02-12', 
    '2021-02-13', '2021-02-15', '2021-02-17', '2021-02-22', '2021-02-23',
    '2021-02-26', '2021-03-10', '2021-03-10', '2021-03-11', '2021-11-19',
    '2021-12-03', '2021-12-05', '2021-12-11', '2021-12-14', '2021-12-16',
    '2021-12-20', '2021-12-23',
    '2022-01-21', '2022-01-23', '2022-10-21', '2022-11-14', '2022-11-15',
    '2022-11-17']),
'lower_window': 0,
'upper_window': 1,
})

holidays = pd.concat((Bok_nal, AI))

In [25]:
from prophet import Prophet

m = Prophet(holidays=holidays)
m.add_country_holidays(country_name='KR').fit(data_5_6)
future = m.make_future_dataframe(periods=4,freq='W') # 주 단위로, 4주 가격 예측 실시
forecast = m.predict(future)

15:03:03 - cmdstanpy - INFO - Chain [1] start processing
15:03:03 - cmdstanpy - INFO - Chain [1] done processing


In [26]:
predict_size_5_6 = forecast[['ds','yhat']]
predict_size_5_6.tail()

,ds,yhat
2196,2022-11-27,4933.0
2197,2022-12-04,4933.0
2198,2022-12-11,4933.0
2199,2022-12-18,4933.0
2200,2022-12-25,4933.0


In [27]:
m = Prophet(holidays=holidays)
m.add_country_holidays(country_name='KR').fit(data_7_8)
future = m.make_future_dataframe(periods=4,freq='W') # 주 단위로, 4주 가격 예측 실시
forecast = m.predict(future)

predict_size_7_8 = forecast[['ds','yhat']]
predict_size_7_8.tail()

15:03:05 - cmdstanpy - INFO - Chain [1] start processing
15:03:05 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
2196,2022-11-27,4720.00236
2197,2022-12-04,4720.00236
2198,2022-12-11,4720.00236
2199,2022-12-18,4720.00236
2200,2022-12-25,4720.00236


In [28]:
from prophet import Prophet

m = Prophet(holidays=holidays)
m.add_country_holidays(country_name='KR').fit(data_9_10)
future = m.make_future_dataframe(periods=4,freq='W') # 주 단위로, 4주 가격 예측 실시
forecast = m.predict(future)

predict_size_9_10 = forecast[['ds','yhat']]
predict_size_9_10.tail()

15:03:07 - cmdstanpy - INFO - Chain [1] start processing
15:03:07 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
2196,2022-11-27,4384.998538
2197,2022-12-04,4384.998538
2198,2022-12-11,4384.998538
2199,2022-12-18,4384.998538
2200,2022-12-25,4384.998538


In [29]:
from prophet import Prophet

m = Prophet(holidays=holidays)
m.add_country_holidays(country_name='KR').fit(data_11)
future = m.make_future_dataframe(periods=4,freq='W') # 주 단위로, 4주 가격 예측 실시
forecast = m.predict(future)

predict_size_11 = forecast[['ds','yhat']]
predict_size_11.tail()

15:03:09 - cmdstanpy - INFO - Chain [1] start processing
15:03:10 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
2196,2022-11-27,3855.903882
2197,2022-12-04,3900.970459
2198,2022-12-11,3969.097499
2199,2022-12-18,4063.739994
2200,2022-12-25,4305.582469


In [30]:
from prophet import Prophet

m = Prophet(holidays=holidays)
m.add_country_holidays(country_name='KR').fit(data_12)
future = m.make_future_dataframe(periods=4,freq='W') # 주 단위로, 4주 가격 예측 실시
forecast = m.predict(future)

predict_size_12 = forecast[['ds','yhat']]
predict_size_12.tail()

15:03:12 - cmdstanpy - INFO - Chain [1] start processing
15:03:13 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
2196,2022-11-27,3590.747942
2197,2022-12-04,3625.462449
2198,2022-12-11,3691.599509
2199,2022-12-18,3785.799926
2200,2022-12-25,4022.875555


In [31]:
from prophet import Prophet

m = Prophet(holidays=holidays)
m.add_country_holidays(country_name='KR').fit(data_13_16)
future = m.make_future_dataframe(periods=4,freq='W') # 주 단위로, 4주 가격 예측 실시
forecast = m.predict(future)

predict_size_13_16 = forecast[['ds','yhat']]
predict_size_13_16.tail()

15:03:15 - cmdstanpy - INFO - Chain [1] start processing
15:03:15 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
2196,2022-11-27,3907.0
2197,2022-12-04,3907.0
2198,2022-12-11,3907.0
2199,2022-12-18,3907.0
2200,2022-12-25,3907.0


In [32]:
dfs = [predict_size_5_6,predict_size_7_8,predict_size_9_10,predict_size_11,predict_size_12,predict_size_13_16]

import functools as ft
df_final = ft.reduce(lambda left,right : pd.merge(left,right, on='ds',how='left'), dfs)

C:\Users\admin\AppData\Local\Temp\ipykernel_28620\3519427148.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'yhat_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_final = ft.reduce(lambda left,right : pd.merge(left,right, on='ds',how='left'), dfs)
C:\Users\admin\AppData\Local\Temp\ipykernel_28620\3519427148.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'yhat_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_final = ft.reduce(lambda left,right : pd.merge(left,right, on='ds',how='left'), dfs)


In [33]:
df_final.tail()

,ds,yhat_x,yhat_y,yhat_x,yhat_y,yhat_x,yhat_y
2196,2022-11-27,4933.0,4720.00236,4384.998538,3855.903882,3590.747942,3907.0
2197,2022-12-04,4933.0,4720.00236,4384.998538,3900.970459,3625.462449,3907.0
2198,2022-12-11,4933.0,4720.00236,4384.998538,3969.097499,3691.599509,3907.0
2199,2022-12-18,4933.0,4720.00236,4384.998538,4063.739994,3785.799926,3907.0
2200,2022-12-25,4933.0,4720.00236,4384.998538,4305.582469,4022.875555,3907.0


In [34]:
col = ['date','5_6호', '7_8호', '9_10호', '11호', '12호', '13_16호']

df_final.columns = col

In [35]:
df_final['평균가격'] = df_final.mean(axis=1)

C:\Users\admin\AppData\Local\Temp\ipykernel_28620\2566952264.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_final['평균가격'] = df_final.mean(axis=1)


In [36]:
df_final = df_final.tail()
df_final

,date,5_6호,7_8호,9_10호,11호,12호,13_16호,평균가격
2196,2022-11-27,4933.0,4720.00236,4384.998538,3855.903882,3590.747942,3907.0,4231.942120
2197,2022-12-04,4933.0,4720.00236,4384.998538,3900.970459,3625.462449,3907.0,4245.238968
2198,2022-12-11,4933.0,4720.00236,4384.998538,3969.097499,3691.599509,3907.0,4267.616318
2199,2022-12-18,4933.0,4720.00236,4384.998538,4063.739994,3785.799926,3907.0,4299.090136
2200,2022-12-25,4933.0,4720.00236,4384.998538,4305.582469,4022.875555,3907.0,4378.909820
